In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoarthritis"
cohort = "GSE93720"

# Input paths
in_trait_dir = "../../input/GEO/Osteoarthritis"
in_cohort_dir = "../../input/GEO/Osteoarthritis/GSE93720"

# Output paths
out_data_file = "../../output/preprocess/Osteoarthritis/GSE93720.csv"
out_gene_data_file = "../../output/preprocess/Osteoarthritis/gene_data/GSE93720.csv"
out_clinical_data_file = "../../output/preprocess/Osteoarthritis/clinical_data/GSE93720.csv"
json_path = "../../output/preprocess/Osteoarthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional response of human synovial fibroblasts to TNF and IL-17A (Microarray Data 2)"
!Series_summary	"The combination of TNF and IL-17A has synergistic effects on the transcription of genes encoding chemokines and cytokines."
!Series_overall_design	"Human fibroblasts derived from 2 individuals with rheumatoid arthritis and 2 individuals with osteoarthritis were stimulated with TNF, IL-17A, or TNF and IL-17A and gene expression was assayed with microarrays."
!Series_overall_design	""
!Series_overall_design	"Microarray Data 2 (n = 48): 1 time point (24 hours), 3 passages, 4 cell lines, 4 stimulations (Baseline; None; TNF; TNF and IL-17A)."
!Series_overall_design	""
!Series_overall_design	"GeneChip™ Human Genome U133 Plus 2.0 Array (""affy_hg_u133_plus_2"" on Bioconductor)."
Sample Characteristics Dictionary:
{0: ['disease: OA', 'disease: RA'], 1: ['donor: OA6', 'donor: RA32', 'donor: OA502', 'donor: RA449'], 2: ['il6: high', 'il6: low'], 3:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Yes, this dataset contains gene expression data from microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For the trait (Osteoarthritis), we can find it in the disease field (key 0)
trait_row = 0  # The "disease: OA" or "disease: RA" records

# No age information is available in the sample characteristics
age_row = None

# No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert the disease value to binary (1 for Osteoarthritis, 0 for other conditions)"""
    if not isinstance(value, str):
        return None
        
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # OA means Osteoarthritis, RA means Rheumatoid Arthritis
    if value.upper() == "OA":
        return 1  # Has Osteoarthritis
    elif value.upper() == "RA":
        return 0  # Does not have Osteoarthritis (has RA instead)
    return None

# No need to define convert_age and convert_gender since these data are not available

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available)
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=None,
        gender_row=gender_row,
        convert_gender=None
    )
    
    # Preview the results
    print("Clinical Data Preview:")
    print(preview_df(clinical_df))
    
    # Save the clinical data to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM3713911': [1.0], 'GSM3713912': [1.0], 'GSM3713913': [1.0], 'GSM3713914': [1.0], 'GSM3713915': [1.0], 'GSM3713916': [1.0], 'GSM3713917': [1.0], 'GSM3713918': [0.0], 'GSM3713919': [0.0], 'GSM3713920': [0.0], 'GSM3713921': [0.0], 'GSM3713922': [0.0], 'GSM3713923': [0.0], 'GSM3713924': [0.0], 'GSM3713925': [0.0], 'GSM3713926': [1.0], 'GSM3713927': [1.0], 'GSM3713928': [1.0], 'GSM3713929': [1.0], 'GSM3713930': [1.0], 'GSM3713931': [1.0], 'GSM3713932': [1.0], 'GSM3713933': [1.0], 'GSM3713934': [0.0], 'GSM3713935': [0.0], 'GSM3713936': [0.0], 'GSM3713937': [0.0], 'GSM3713938': [0.0], 'GSM3713939': [0.0], 'GSM3713940': [0.0], 'GSM3713941': [0.0], 'GSM3713942': [1.0], 'GSM3713943': [1.0], 'GSM3713944': [1.0], 'GSM3713945': [0.0], 'GSM3713946': [0.0], 'GSM3713947': [0.0], 'GSM3713948': [1.0], 'GSM3713949': [1.0], 'GSM3713950': [1.0], 'GSM3713951': [0.0], 'GSM3713952': [0.0], 'GSM3713953': [0.0], 'GSM3713954': [1.0], 'GSM3713955': [0.0], 'GSM3713956': [1.0], 'GSM371395

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Osteoarthritis/GSE93720/GSE93720_series_matrix.txt.gz


Gene data shape: (54675, 48)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# This step doesn't require code implementation, just review and determination
# These are Affymetrix probe IDs (e.g., "1007_s_at"), not human gene symbols
# They follow the standard Affymetrix format with "_at" suffix
# These need to be mapped to human gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for gene mapping
# The 'ID' column in gene_annotation corresponds to the probe identifiers in gene_data
# The 'Gene Symbol' column contains the human gene symbols we want to map to
prob_col = 'ID'  # This is the probe identifier column
gene_col = 'Gene Symbol'  # This is the gene symbol column

# 2. Get gene mapping dataframe using helper function 
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(preview_df(gene_mapping, n=5))

# 3. Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data after mapping, shape: {gene_data.shape}")
print("First few gene symbols:")
print(list(gene_data.index[:10]))

# Normalize gene symbols to handle synonyms and duplicates
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data after normalization, shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(list(gene_data.index[:10]))

# Save the gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Mapped gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (45782, 2)
Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}


Gene expression data after mapping, shape: (21278, 48)
First few gene symbols:
['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Gene expression data after normalization, shape: (19845, 48)
First few normalized gene symbols:
['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Mapped gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE93720.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Gene data was already normalized and saved in step 6

# 2. Link the clinical and genetic data
# First, load the clinical data saved in step 2
clinical_df = pd.read_csv(out_clinical_data_file)

# Print column names to inspect
print("Clinical data columns:", clinical_df.columns.tolist())

# Set up clinical features dataframe in the format expected by geo_link_clinical_genetic_data
# The first column is the sample ID column, and any remaining column contains the trait data
clinical_features_df = pd.DataFrame(clinical_df)
data_columns = [col for col in clinical_df.columns if col != clinical_df.columns[0]]
if len(data_columns) > 0:
    # Rename the data column to the trait name
    clinical_features_df = clinical_features_df.rename(columns={data_columns[0]: trait})
else:
    # If there's no data column, something is wrong with the clinical data
    raise ValueError("Clinical data does not contain trait information")

# Set the index to the first column (sample IDs) and transpose
clinical_features_df = clinical_features_df.set_index(clinical_features_df.columns[0])
clinical_features_df = clinical_features_df.T

print("Clinical data shape:", clinical_features_df.shape)
print("Clinical data preview:", preview_df(clinical_features_df))

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Evaluate bias in trait and demographic features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct final quality validation
note = "This dataset contains human synovial fibroblasts derived from individuals with rheumatoid arthritis and osteoarthritis, stimulated with different conditions."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Clinical data columns: ['GSM3713911', 'GSM3713912', 'GSM3713913', 'GSM3713914', 'GSM3713915', 'GSM3713916', 'GSM3713917', 'GSM3713918', 'GSM3713919', 'GSM3713920', 'GSM3713921', 'GSM3713922', 'GSM3713923', 'GSM3713924', 'GSM3713925', 'GSM3713926', 'GSM3713927', 'GSM3713928', 'GSM3713929', 'GSM3713930', 'GSM3713931', 'GSM3713932', 'GSM3713933', 'GSM3713934', 'GSM3713935', 'GSM3713936', 'GSM3713937', 'GSM3713938', 'GSM3713939', 'GSM3713940', 'GSM3713941', 'GSM3713942', 'GSM3713943', 'GSM3713944', 'GSM3713945', 'GSM3713946', 'GSM3713947', 'GSM3713948', 'GSM3713949', 'GSM3713950', 'GSM3713951', 'GSM3713952', 'GSM3713953', 'GSM3713954', 'GSM3713955', 'GSM3713956', 'GSM3713957', 'GSM3713958']
Clinical data shape: (47, 1)
Clinical data preview: {1.0: [1.0, 1.0, 1.0, 1.0, 1.0]}
Linked data shape before handling missing values: (49, 19892)
Linked data shape after handling missing values: (1, 47)
Quartiles for 'Osteoarthritis':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The dist